# Object Detection

# Author: Ian Arakkal

Code Adapted From: https://towardsdatascience.com/object-detection-with-tensorflow-model-and-opencv-d839f3e42849

In [ ]:
from scipy.spatial import distance as dist
import tensorflow_hub as hub
import cv2
import glob
import numpy
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os

In [ ]:
detector = hub.load("https://tfhub.dev/tensorflow/efficientdet/lite4/detection/1")

In [ ]:
labels = pd.read_csv('labels.csv', sep=';', index_col='ID')
labels = labels['SUPER CATEGORY']

def detect_objects(input_filepath, output_filepath, width=1028, height=1028, show=False):
  # load data and preprocessing
  img = cv2.imread(input_filepath)
  if width is None or height == None:
    inp = img
  else:
    inp = cv2.resize(img, (width, height))
  rgb = cv2.cvtColor(inp, cv2.COLOR_BGR2RGB)
  rgb_tensor = tf.convert_to_tensor(rgb, dtype = tf.uint8)
  rgb_tensor = tf.expand_dims(rgb_tensor, 0)

  # run model
  boxes, scores, classes, num_detection = detector(rgb_tensor)

  # post processing
  pred_labels = classes.numpy().astype('int')[0] 
  pred_labels = [labels[i] for i in pred_labels]
  pred_boxes = boxes.numpy()[0].astype('int')
  pred_scores = scores.numpy()[0]

  # draw boxes on the figure
  output_image = rgb
  for score, (ymin, xmin, ymax, xmax), label in zip(pred_scores, pred_boxes, pred_labels):
    if score < 0.5:
      continue

    # draw box    
    score_txt = f'{100 * round(score)}%'
    output_image = cv2.rectangle(output_image, (xmin, ymax),(xmax, ymin),(0,255,0),2)

    # put text
    font = cv2.FONT_HERSHEY_SIMPLEX
    output_image = cv2.putText(output_image, label, (xmin, ymax-10), font, 1.5, (255,0,0), 2, cv2.LINE_AA)
    output_image = cv2.putText(output_image, score_txt, (xmax, ymax-10), font, 1.5, (255,0,0), 2, cv2.LINE_AA)
      
  # save figure to the specified path
  cv2_output_image = cv2.cvtColor(output_image, cv2.COLOR_RGB2BGR)
  cv2.imwrite(output_filepath, cv2_output_image)
  
  # show figure if "show" is true
  if show:
    plt.figure(figsize=(10,10))
    plt.imshow(output_image)

In [ ]:
# Running in batch mode
input_folder = "H:\School\Semester 14 (Fall 21)\DS 440\Testing" #directory containing images
filenames = os.listdir(input_folder)
filenames = [filename for filename in filenames if filename.endswith(".png")] #if image type is not .jpg, replace with appropirate photo type
print(filenames)

In [ ]:
output_folder = "H:\School\Semester 14 (Fall 21)\DS 440\TestProc" #destination folder for images with object detected
os.makedirs(output_folder, exist_ok=True)

In [ ]:
for index, filename in enumerate(filenames, 1):
  print("Processing {} / {} - {}".format(index, len(filenames), filename))
  detect_objects(
    input_filepath = os.path.join(input_folder, filename),
    output_filepath = os.path.join(output_folder, filename),
    show=False, # you can turn this on if needed  
  )